In [1]:
from pymongo import MongoClient
import requests
import time

In [14]:
# connect to the hosted MongoDB instance
client = MongoClient()
db = client['test_database']
collection = db['test_table']
collection.create_index([('hash', 1)], unique=True)

u'hash_1'

In [15]:
def collect_data(count, offset):
    """
    DESCRIPTION:
        Collects blockchain transaction data from etherchain.org API
        https://etherchain.org/documentation/api/
    
    INPUT:
        - offset: the number of txs to skip
        - count: the number of txs to return (max 100 blocks per request)
        - sleeptime: number of seconds to sleep between api requests
    OUTPUT:
        - stores each blockchain transaction as a document in a MongoDB collection
    """
    
    BASE_URL = 'https://etherchain.org/api/txs/{}/{}'.format(offset, count)
    r = requests.get(BASE_URL)

    if r.status_code != 200:
        print('status code = {}'.format(r.status_code))

    else:
    # store each transaction block in our mongo database
        r = convert_price(r.json())

        for row in range(count):
            try:

                collection.insert_one(r['data'][row])

            except Exception as e:
                #print e
                #print(r['data'][row])
                pass
        
def convert_price(json_dict):
    
    for tx in json_dict['data']:
        tx['amount'] = float(tx['amount'])
        tx['price'] = float(tx['price'])
    
    return json_dict

def call_api():
    
    offset = 0
    x = 0
    while x < 10:
        collect_data(100, offset)
        offset += 200
        #print(offset)
        if offset % 1000 == 0:
            print('offset: {}'.format(offset))
           
        time.sleep(5)
        x += 1

In [18]:
collection.count()

4341

In [17]:
call_api()

offset: 1000
offset: 2000


In [8]:
collection.count()

2831

In [7]:
collection.find_one()

{u'_id': ObjectId('59c470ae66a1619ccb3ab917'),
 u'accountNonce': u'109003',
 u'amount': 7.437130289163643e+17,
 u'blockHash': u'0xe1ea5bfdce01981c63d27c7df4fe80bd7ef2f6b35cf8e711daccb22814291770',
 u'block_id': 4300376,
 u'gasLimit': 50000,
 u'gasUsed': 21000,
 u'hash': u'0xffa830b0ec602cbed484ceba2a4b12a0c581eeec3ea194ecf7a7d241859a275d',
 u'isContractTx': None,
 u'newContract': 0,
 u'parentHash': u'0xffa830b0ec602cbed484ceba2a4b12a0c581eeec3ea194ecf7a7d241859a275d',
 u'price': 21000000000.0,
 u'recipient': u'0x0dc0bfa68ebf8a0062bce80a5955a949f731cf85',
 u'sender': u'0x90e4be5cd24970a87e23e281f3ad40a89558b433',
 u'time': u'2017-09-22T02:08:07.000Z',
 u'txIndex': None,
 u'type': u'tx'}

In [19]:
len(collection.distinct('hash'))

4341